In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp helpers

In [ ]:
#| export
import copy

In [ ]:
#| export

class ARCHelper:
    def __init__(self, arc):
        self.arc = arc
        self.train_input_dims = []
        self.train_output_dims = []
        self.test_input_dims = []

        self._extract_dimensions()

    def _extract_dimensions(self):
        # Extract dimensions for train inputs and outputs
        for item in self.arc.get('train', []):
            input_dims = self._get_dimensions(item['input'])
            output_dims = self._get_dimensions(item['output'])
            self.train_input_dims.append(input_dims)
            self.train_output_dims.append(output_dims)

        # Extract dimensions for test inputs
        for item in self.arc.get('test', []):
            input_dims = self._get_dimensions(item['input'])
            self.test_input_dims.append(input_dims)

    def _get_dimensions(self, array):
        return (len(array), len(array[0]) if array else 0)

    def get_dimensions(self, primary_key, nested_key):
        if primary_key not in ['train', 'test']:
            raise ValueError("Primary key must be 'train' or 'test'")
        if nested_key not in ['input', 'output']:
            raise ValueError("Nested key must be 'input' or 'output'")

        if primary_key == 'train':
            if nested_key == 'input':
                return self.train_input_dims
            elif nested_key == 'output':
                return self.train_output_dims
        elif primary_key == 'test':
            if nested_key == 'input':
                return self.test_input_dims
            # The test data does not have 'output' dimensions
            else:
                raise ValueError("'output' key is not available for 'test'")

    def get_number_of_train_elements(self):
        return len(self.arc.get('train', []))

    def get_train_input_width(self, index):
        return self.train_input_dims[index][1]

    def get_train_input_height(self, index):
        return self.train_input_dims[index][0]

    def get_train_output_width(self, index):
        return self.train_output_dims[index][1]

    def get_train_output_height(self, index):
        return self.train_output_dims[index][0]

    def compare_with_train_output(self, index, array):
        if index >= len(self.arc.get('train', [])):
            raise IndexError("Index out of range for train data")
        
        train_output = self.arc['train'][index]['output']
        return train_output == array

    def metric(self, index, array):
        if index >= len(self.arc.get('train', [])):
            raise IndexError("Index out of range for train data")

        provided_height, provided_width = self._get_dimensions(array)
        train_output_height = self.get_train_output_height(index)
        train_output_width = self.get_train_output_width(index)

        height_ratio = provided_height / train_output_height
        width_ratio = provided_width / train_output_width

        return (height_ratio + width_ratio) / 2

    def add_or_remove_column(self, array, value, num_columns=1, add=True):
        for row in array:
            if add:
                row.extend([value] * num_columns)
            else:
                if len(row) >= num_columns:
                    row = row[:-num_columns]
                else:
                    row = []
        return array

    def add_or_remove_row(self, array, value, num_rows=1, add=True):
        if add:
            for _ in range(num_rows):
                array.append([value] * (len(array[0]) if array else 1))
        else:
            for _ in range(num_rows):
                if array:
                    array.pop()
        return array

    def clone_train_input(self, index):
        if index >= len(self.arc.get('train', [])):
            raise IndexError("Index out of range for train data")
        return copy.deepcopy(self.arc['train'][index]['input'])


    def get_dimensions_array(self, index):
        if index >= len(self.arc.get('train', [])):
            raise IndexError("Index out of range for train data")

        input_height, input_width = self.train_input_dims[index]
        output_height, output_width = self.train_output_dims[index]

        return [input_width, input_height, output_width, output_height]
    
    
    def get_dimensions_provided_array(self, input_array, output_array):
        input_height, input_width = self._get_dimensions(input_array)
        output_height, output_width = self._get_dimensions(output_array)

        return [input_width, input_height, output_width, output_height]


In [ ]:


# Example usage
arc = {
    'test': [{'input': [[7, 0, 7], [7, 0, 7], [7, 7, 0]]}],
    'train': [
        {'input': [[0, 7, 7], [7, 7, 7], [0, 7, 7]], 'output': [[0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 0, 0, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 7, 7, 0, 7, 7, 0, 7, 7], [7, 7, 7, 7, 7, 7, 7, 7, 7], [0, 7, 7, 0, 7, 7, 0, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 0, 0, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7]]},
        {'input': [[4, 0, 4], [0, 0, 0], [0, 4, 0]], 'output': [[4, 0, 4, 0, 0, 0, 4, 0, 4], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 4, 0, 0, 0, 0, 0, 4, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 4, 0, 4, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 4, 0, 0, 0, 0]]},
        {'input': [[0, 0, 0], [0, 0, 2], [2, 0, 2]], 'output': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 2, 0, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 0, 2], [2, 0, 2, 0, 0, 0, 2, 0, 2]]},
        {'input': [[6, 6, 0], [6, 0, 0], [0, 6, 6]], 'output': [[6, 6, 0, 6, 6, 0, 0, 0, 0], [6, 0, 0, 6, 0, 0, 0, 0, 0], [0, 6, 6, 0, 6, 6, 0, 0, 0], [6, 6, 0, 0, 0, 0, 0, 0, 0], [6, 0, 0, 0, 0, 0, 0, 0, 0], [0, 6, 6, 0, 0, 0, 0, 0, 0], [0, 0, 0, 6, 6, 0, 6, 6, 0], [0, 0, 0, 6, 0, 0, 6, 0, 0], [0, 0, 0, 0, 6, 6, 0, 6, 6]]},
        {'input': [[2, 2, 2], [0, 0, 0], [0, 2, 2]], 'output': [[2, 2, 2, 2, 2, 2, 2, 2, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 2, 2, 0, 2, 2, 0, 2, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 2, 2, 2, 2, 2, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 2, 2, 0, 2, 2]]}
    ]
}

arc_helper = ARCHelper(arc)
print("Number of Train Elements:", arc_helper.get_number_of_train_elements())
print("Train Input Width at Index 0:", arc_helper.get_train_input_width(0))
print("Train Input Height at Index 0:", arc_helper.get_train_input_height(0))
print("Train Output Width at Index 0:", arc_helper.get_train_output_width(0))
print("Train Output Height at Index 0:", arc_helper.get_train_output_height(0))

# Compare with train output
provided_array = [[0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 0, 0, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 7, 7, 0, 7, 7, 0, 7, 7], [7, 7, 7, 7, 7, 7, 7, 7, 7], [0, 7, 7, 0, 7, 7, 0, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 0, 0, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7]]
is_equal = arc_helper.compare_with_train_output(0, provided_array)
print("Provided array is equal to train output at index 0:", is_equal)

# Metric comparison with train output
metric_value = arc_helper.metric(0, provided_array)
print("Metric value for provided array compared with train output at index 0:", metric_value)

# Add 2 columns
modified_array = arc_helper.add_or_remove_column(provided_array, value=1, num_columns=2, add=True)
print("Array after adding 2 columns:", modified_array)

# Remove 1 column
modified_array = arc_helper.add_or_remove_column(provided_array, value=1, num_columns=1, add=False)
print("Array after removing 1 column:", modified_array)

# Add 2 rows
modified_array = arc_helper.add_or_remove_row(provided_array, value=1, num_rows=2, add=True)
print("Array after adding 2 rows:", modified_array)

# Remove 1 row
modified_array = arc_helper.add_or_remove_row(provided_array, value=1, num_rows=1, add=False)
print("Array after removing 1 row:", modified_array)



# Provided array of size 2 by 3
provided_array = [
    [1, 2, 3],
    [4, 5, 6]
]

# Example index to compare with train output (using index 0 for demonstration)
index = 0

# Metric comparison with train output
metric_value = arc_helper.metric(index, provided_array)
print("Metric value for provided array compared with train output at index 0:", metric_value)

# Get dimensions array for train data at index 0
dimensions_array = arc_helper.get_dimensions_array(index)
print("Dimensions array for train data at index 0:", dimensions_array)



# Provided array of size 2 by 3
provided_input_array = [
    [1, 2, 3],
    [4, 5, 6]
]

# Provided output array of size 4 by 5
provided_output_array = [
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1]
]

# Get dimensions array for provided arrays
dimensions_provided_array = arc_helper.get_dimensions_provided_array(provided_input_array, provided_output_array)
print("Dimensions array for provided input and output arrays:", dimensions_provided_array)



Number of Train Elements: 5
Train Input Width at Index 0: 3
Train Input Height at Index 0: 3
Train Output Width at Index 0: 9
Train Output Height at Index 0: 9
Provided array is equal to train output at index 0: True
Metric value for provided array compared with train output at index 0: 1.0
Array after adding 2 columns: [[0, 0, 0, 0, 7, 7, 0, 7, 7, 1, 1], [0, 0, 0, 7, 7, 7, 7, 7, 7, 1, 1], [0, 0, 0, 0, 7, 7, 0, 7, 7, 1, 1], [0, 7, 7, 0, 7, 7, 0, 7, 7, 1, 1], [7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 1], [0, 7, 7, 0, 7, 7, 0, 7, 7, 1, 1], [0, 0, 0, 0, 7, 7, 0, 7, 7, 1, 1], [0, 0, 0, 7, 7, 7, 7, 7, 7, 1, 1], [0, 0, 0, 0, 7, 7, 0, 7, 7, 1, 1]]
Array after removing 1 column: [[0, 0, 0, 0, 7, 7, 0, 7, 7, 1, 1], [0, 0, 0, 7, 7, 7, 7, 7, 7, 1, 1], [0, 0, 0, 0, 7, 7, 0, 7, 7, 1, 1], [0, 7, 7, 0, 7, 7, 0, 7, 7, 1, 1], [7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 1], [0, 7, 7, 0, 7, 7, 0, 7, 7, 1, 1], [0, 0, 0, 0, 7, 7, 0, 7, 7, 1, 1], [0, 0, 0, 7, 7, 7, 7, 7, 7, 1, 1], [0, 0, 0, 0, 7, 7, 0, 7, 7, 1, 1]]
Array after add

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()